## **15-03 양방향 LSTM과 어텐션 메커니즘(BiLSTM with Attention mechanism)**

단뱡항 LSTM으로 텍스트 분류를 수행할 수도 있지만 때로는 **양방향 LSTM** 을 사용하는 것이 더 강력합니다. 여기에 추가적으로 **어텐션 메커니즘** 을 사용할 수도 있습니다. 양방향 LSTM과 어텐션 메커니즘으로 IMDB 리뷰 감성 분류하기를 수행해봅시다.



### **1.IMDB 리뷰 데이터 전처리하기**

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

IMDB 리뷰 데이터는 앞서 텍스트 분류하기 챕터에서 다룬 바 있으므로 데이터에 대한 상세 설명은 생략합니다. 최대 단어 개수를 10,000으로 제한하고 훈련 데이터와 테스트 데이터를 받아옵니다.

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [4]:
y_train.shape, y_train

((25000,), array([1, 0, 0, ..., 0, 1, 0]))

In [34]:
# imdb.get_word_index()

훈련 데이터와 이에 대한 레이블이 각각 X_train, y_train에 테스트 데이터와 이에 대한 레이블이 각각 X_test, y_test에 저장되었습니다. IMDB 리뷰 데이터는 이미 정수 인코딩이 된 상태므로 남은 전처리는 패딩뿐입니다. 리뷰의 최대 길이와 평균 길이를 확인해봅시다.

In [6]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


리뷰의 최대 길이는 2,494이며 리뷰의 평균 길이는 약 238로 확인됩니다. 평균 길이보다는 조금 크게 데이터를 패딩하겠습니다.

In [7]:
###########################################################################
# Colab GPU 사용시 Error: 반드시 우측 Padding 되어야 함 (CuDNN)
###########################################################################
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

In [8]:
X_train.shape, X_train

((25000, 500),
 array([[   1,   14,   22, ...,    0,    0,    0],
        [   1,  194, 1153, ...,    0,    0,    0],
        [   1,   14,   47, ...,    0,    0,    0],
        ...,
        [   1,   11,    6, ...,    0,    0,    0],
        [   1, 1446, 7079, ...,    0,    0,    0],
        [   1,   17,    6, ...,    0,    0,    0]], dtype=int32))

In [9]:
X_test.shape, X_test

((25000, 500),
 array([[   1,  591,  202, ...,    0,    0,    0],
        [   1,   14,   22, ...,    0,    0,    0],
        [  33,    6,   58, ...,    9,   57,  975],
        ...,
        [   1,   13, 1408, ...,    0,    0,    0],
        [   1,   11,  119, ...,    0,    0,    0],
        [   1,    6,   52, ...,    0,    0,    0]], dtype=int32))

훈련용 리뷰와 테스트용 리뷰의 길이가 둘 다 500이 되었습니다.

### **2.바다나우 어텐션(Bahdanau Attention)**


여기서 사용할 어텐션은 바다나우 어텐션(Bahdanau attention)입니다. 이를 이해하기 위해 앞서 배운 가장 쉬운 어텐션이었던 닷 프로덕트 어텐션과 어텐션 스코어 함수의 정의를 상기해봅시다.

어텐션 스코어 함수란 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수를 말합니다. 그리고 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)이었습니다. 다음은 닷 프로덕트 어텐션의 어텐션 스코어 함수를 보여줍니다.

##### $score(query, key) = query^T key$
  

바다나우 어텐션은 아래와 같은 어텐션 스코어 함수를 사용합니다.
  
##### $score(query, key) = V^T tanh(W_1 key + W_2 query)$
  
  
이 어텐션 스코어 함수를 사용하여 어텐션 메커니즘을 구현하면 됩니다. 그런데 텍스트 분류에서 어텐션 메커니즘을 사용하는 이유는 무엇일까요? RNN의 마지막 은닉 상태는 예측을 위해 사용됩니다. 그런데 이 RNN의 마지막 은닉 상태는 몇 가지 유용한 정보들을 손실한 상태입니다. 그래서 RNN이 time step을 지나며 손실했던 정보들을 다시 참고하고자 합니다.

이는 다시 말해 RNN의 모든 은닉 상태들을 다시 한 번 참고하겠다는 것입니다. 그리고 이를 위해서 어텐션메커니즘을 사용합니다.

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.utils import register_keras_serializable  # Import for serialization

@register_keras_serializable()  #  BahdanauAttention이라는 사용자 정의 클래스가 직렬화(Serialization) 및 역직렬화(Deserialization) 과정에서 인식되지 않아서 발생한 오류 해결용
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### **3.양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)**

In [12]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

이제 모델을 설계해보겠습니다. 여기서는 케라스의 함수형 API를 사용합니다. 우선 입력층과 임베딩층을 설계합니다.

In [13]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10,000개의 단어들을 128차원의 벡터로 임베딩하도록 설계하였습니다. 이제 양방향 LSTM을 설계합니다. 단, 여기서는 양방향 LSTM을 두 층을 사용하겠습니다. 우선, 첫번째 층입니다. 두번째 층을 위에 쌓을 예정이므로 return_sequences를 True로 해주어야 합니다.

In [14]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

두번째 층을 설계합니다. 상태를 리턴받아야 하므로 return_state를 True로 해주어야 합니다.

In [15]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

각 상태의 크기(shape)를 출력해보겠습니다.

In [16]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장하고, 역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장합니다.

각 은닉 상태나 셀 상태의 경우에는 128차원을 가지는데, lstm의 경우에는 (500 × 128)의 크기를 가집니다. foward 방향과 backward 방향이 연결된 hidden state벡터가 모든 시점에 대해서 존재함을 의미합니다.

양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

In [17]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [18]:
state_h, state_c

(<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_9>,
 <KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_10>)

어텐션 메커니즘에서는 은닉 상태를 사용합니다. 이를 입력으로 컨텍스트 벡터(context vector)를 얻습니다.

In [19]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'bahdanau_attention' (of type BahdanauAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [20]:
attention, context_vector, attention_weights

(<BahdanauAttention name=bahdanau_attention, built=True>,
 <KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_13>,
 <KerasTensor shape=(None, 500, 1), dtype=float32, sparse=False, name=keras_tensor_14>)

컨텍스트 벡터를 밀집층(dense layer)에 통과시키고, 이진 분류이므로 최종 출력층에 1개의 뉴런을 배치하고, 활성화 함수로 시그모이드 함수를 사용합니다.

In [21]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [22]:
dense1, dropout, output, model

(<KerasTensor shape=(None, 20), dtype=float32, sparse=False, name=keras_tensor_15>,
 <KerasTensor shape=(None, 20), dtype=float32, sparse=False, name=keras_tensor_16>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_17>,
 <Functional name=functional, built=True>)

옵티마이저로 아담 옵티마이저 사용하고, 모델을 컴파일합니다.

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 500, 128)  │  1,280,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 500)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 500, 128)  │     98,816 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ [(None, 500,      │     98,816 │ bidirectional[0]… │
│ (Bidirectional)     │ 128), (None, 64), │            │ not_equal[0][0]   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bahdanau_attention  │ [(None, 128),     │     16,577 │ bidirectional_1[… │
│ (BahdanauAttention) │ (None, 500, 1)]   │            │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 20)        │      2,580 │ bahdanau_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 20)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         21 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,496,810 (5.71 MB)

 Trainable params: 1,496,810 (5.71 MB)

 Non-trainable params: 0 (0.00 B)

시그모이드 함수를 사용하므로 손실 함수로 binary_crossentropy를 사용하였습니다. 이제 모델을 훈련하겠습니다.  
검증 데이터로 테스트 데이터를 사용하여 에포크가 끝날 때마다 테스트 데이터에 대한 정확도를 출력하도록 하였습니다.

In [25]:
%%time
# colab - Epoch 3/3
# 98/98 [==============================] - 1170s 12s/step - loss: 0.1934 - accuracy: 0.9379 - val_loss: 0.3021 - val_accuracy: 0.8785
# CPU times: user 1h 42min 41s, sys: 8min 55s, total: 1h 51min 37s
# Wall time: 59min 37s

# colab GPU : 2024.08 (Padding 이 반드시 padding='post'로 우측 padding이 되어야 함)
# Epoch 3/3
# 98/98 ━━━━━━━━━━━━━━━━━━━━ 35s 191ms/step - accuracy: 0.9239 - loss: 0.2146 - val_accuracy: 0.8803 - val_loss: 0.3061
# CPU times: user 54.3 s, sys: 5.76 s, total: 1min
# Wall time: 1min 30s

history = model.fit(X_train, y_train, epochs=3, batch_size=256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 189ms/step - accuracy: 0.6464 - loss: 0.6055 - val_accuracy: 0.8728 - val_loss: 0.3067
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 36s 180ms/step - accuracy: 0.9062 - loss: 0.2651 - val_accuracy: 0.8768 - val_loss: 0.2984
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 20s 180ms/step - accuracy: 0.9333 - loss: 0.1947 - val_accuracy: 0.8801 - val_loss: 0.3029
CPU times: user 48.5 s, sys: 6.22 s, total: 54.7 s
Wall time: 1min 25s


In [26]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.8783 - loss: 0.3037

 테스트 정확도: 0.8787


In [37]:
## 정수 인코딩 시퀀스를 문장으로 확인해 보기
# 1. 데이터 로드
# vocab_size = 10000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# 2. 단어 인덱스 로드
word_index = imdb.get_word_index()

# 3. index → word 매핑 딕셔너리 생성
# Keras는 사전에 1~3번까지의 인덱스를 reserved token으로 사용하므로 +3 offset 필요
index_to_word = {index + 3: word for word, index in word_index.items()}
index_to_word[0] = "<PAD>"
index_to_word[1] = "<START>"
index_to_word[2] = "<UNK>"
index_to_word[3] = "<UNUSED>"

# 4. 디코딩 함수 정의
def decode_review(encoded_review):
    return ' '.join([index_to_word.get(i, '?') for i in encoded_review])

# 5. 첫 번째 훈련 샘플 디코딩
decoded = decode_review(X_test[0])
print("디코딩된 리뷰:\n")
print(decoded)


디코딩된 리뷰:

<START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss
